# Example notebook to run the chatbot using OpenAI GPT3.5 turbo with an AI Key

Install prerequisites:

In [1]:
!pip install -r requirements.txt

In [2]:
import json
import os
import textwrap
from pathlib import Path
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


In [3]:
# set my personal temporary API key for the sake of this technical test
with open(Path('config/openai-config.json')) as json_file:
    os.environ['OPENAI_API_KEY'] = json.load(json_file)['OPENAI_API_KEY']

Preprocess text files:

In [4]:
# load and process the text files

loader = DirectoryLoader('./data/text/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
len(documents)

17

In [5]:
# splitting text into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

32

In [6]:
# chunk example
texts[22]

Document(page_content="2. Cochez le bouton vert pour activer la fonctionnalité de groupes utilisateurs.\n\n3. Créez les noms de vos groupes.\n\n4. Sélectionnez les utilisateurs que vous souhaitez ajouter à un groupe.\n\n5. Choisissez le groupe dans lequel vous souhaitez ajouter vos utilisateurs.\n\nOption 2 :\n\nSi vous préférez automatiser la création de groupes via le SSO (Single Sign-On), veuillez contacter votre CSM (Customer Success Manager) pour convenir d'un rendez-vous avec notre équipe technique.", metadata={'source': 'data/text/Créer des groupes utilisateurs.txt'})

In [7]:
# download emgeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [8]:
# embed and store the texts
# supplying a presist_dictectory will store the embeddings on disk
presist_directory = './data/db'

# here the new embeddings being used
embeddings = embeddings

vectordb = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=presist_directory)

In [9]:
# set retreiver
retriever = vectordb.as_retriever()

# test retreiver
docs = retriever.get_relevant_documents("WeLoop est quoi ?")
len(docs)

4

In [10]:
# document example
docs[0]

Document(page_content="Titre : WeLoop c'est quoi ?\n\nDescription: WeLoop est une plateforme Saas basée sur la communauté et l'expérience utilisateur, proposant une solution comportant deux interfaces pour les utilisateurs finaux et les administrateurs.\n\nInterface Utilisateurs (Widget):\n- L'interface pour les utilisateurs finaux s'intègre facilement à vos applications métier.\n\nInterface Administrateurs (Back Office):\n- Les administrateurs de la solution ont accès à une interface spécifique pour gérer les projets et les interactions.\n- Deux niveaux d'administrateurs existent :\n  1. Les Super Admin qui ont accès à tous les projets au sein de l'organisation.\n  2. Les Admin qui sont les responsables opérationnels d'un ou plusieurs projets.", metadata={'source': 'data/text/WeLoop c_est quoi _.txt'})

In [11]:
# create the chain to answer quetions

llm = ChatOpenAI(temperature=0.0) # temperature is 0 to avoid irrelevant responses

qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [12]:
# existing prompt template
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [13]:
# set custom WeLoop prompt template
qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template = """"
Tu permettes à un utilisateur poser des questions concernant l'outil Weloop. Si une ressource pertinente existe tu dois être proposé à l'utilisateur sinon tu chercheras à comprendre le problème ou la question de l'utilisateur afin de formuler un résumé à destination du support client Weloop.
----------------
{context}
"""

In [14]:
# cite sources

def wrap_text_preserve_newlines(text, width=110):
    # split the input text into lines based on newline characters
    lines = text.split('\n')

    # wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# process response
def process_llm_response(llm_response):
    # response
    print(wrap_text_preserve_newlines(llm_response['result']))
    # sources
    print('\n\nSources:')
    for source in llm_response['source_documents']:
        print(source.metadata['source'])

In [15]:
# full example
query = "Comment utiliser WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pour utiliser WeLoop, vous devez suivre les étapes suivantes :

1. Créer un compte : Rendez-vous sur le site de WeLoop et créez un compte en fournissant les informations
nécessaires.

2. Configurer votre espace de travail : Une fois connecté, vous pouvez configurer votre espace de travail en
ajoutant des projets, des utilisateurs et en personnalisant les paramètres selon vos besoins.

3. Intégrer l'interface utilisateur (Widget) : Pour permettre aux utilisateurs finaux d'interagir avec WeLoop,
vous devez intégrer l'interface utilisateur (Widget) dans vos applications métier. Cela peut être fait en
utilisant le code fourni par WeLoop.

4. Créer des groupes utilisateurs : Vous pouvez créer des groupes utilisateurs en fonction de différents
critères tels que la zone géographique, l'usage de l'application, le département de l'entreprise, etc. Cela
facilitera les interactions entre les utilisateurs partageant des thématiques communes.

5. Collecter les feedbacks des utilisateurs : Les utili

In [16]:
# example for irrelevant prompts
query = "Quelle est la couleur officielle de WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

La couleur officielle de WeLoop est le bleu.


Sources:
data/text/Customizer votre widget.txt
data/text/WeLoop c_est quoi _.txt
data/text/WeLoop c_est quoi _.txt
data/text/Créer des groupes utilisateurs.txt


In [17]:
# example for irrelevant prompts
query = "Quel est l'animal officielle de WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

WeLoop n'a pas d'animal officiel. C'est une plateforme Saas basée sur la communauté et l'expérience
utilisateur, et n'est pas associée à un animal spécifique.


Sources:
data/text/WeLoop c_est quoi _.txt
data/text/Customizer votre widget.txt
data/text/Identification de tous les utilisateurs du Widget !.txt
data/text/WeLoop c_est quoi _.txt


In [19]:
# example for irrelevant prompts
query = "Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Voici les étapes pour personnaliser l'interface utilisateur avec WeLoop :

1. Accédez à votre espace de travail WeLoop.
2. Rendez-vous dans le module "Customize".
3. Configurez l'interface utilisateur en accord avec l'identité visuelle de votre projet ou de votre
entreprise.
4. Vous pouvez modifier les couleurs, les polices, les logos et les images pour correspondre à votre marque.
5. Créez les messages à diffuser à chaque étape du parcours utilisateur.
6. Vous pouvez personnaliser les messages de bienvenue, les notifications, les invitations et les rappels.
7. Assurez-vous de sauvegarder vos modifications.
8. Une fois les modifications enregistrées, l'interface utilisateur sera personnalisée selon vos préférences.

Grâce à WeLoop, vous pouvez bénéficier d'une personnalisation complète de l'interface utilisateur, vous
permettant ainsi de promouvoir votre propre marque tout en offrant une expérience utilisateur cohérente et
engageante.


Sources:
data/text/Customizer votre widget.txt
da